In [22]:
from flask import Flask, render_template, redirect, jsonify
from sqlalchemy import create_engine

import sqlalchemy as db

connection_string = 'postgres:Adgjmpt1@postresql.cfwwjixawhdl.us-east-2.rds.amazonaws.com:5432/RecipeGenerator'
engine = db.create_engine(f'postgresql://{connection_string}', echo=True)
connection = engine.connect()
metadata = db.MetaData()

2020-12-01 15:30:37,995 INFO sqlalchemy.engine.base.Engine select version()
2020-12-01 15:30:37,996 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 15:30:38,065 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-12-01 15:30:38,065 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 15:30:38,134 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-01 15:30:38,135 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 15:30:38,169 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-01 15:30:38,170 INFO sqlalchemy.engine.base.Engine {}
2020-12-01 15:30:38,204 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-12-01 15:30:38,205 INFO sqlalchemy.engine.base.Engine {}


In [31]:
table = db.Table('recipe_list', metadata, autoload = True, autoload_with = engine)
query = db.select(
    [
        table.c.id,
        table.c.name,
        table.c.description,
        table.c.category,
        table.c.subcategory,
        table.c.prep_time,
        table.c.total_time,
        table.c.calories,
        table.c.hf_id,
        table.c.tip,
        table.c.image_url
    ]
)
result = connection.execute(query).fetchall()

for ea in result:
    print(ea)



2020-12-01 15:32:52,281 INFO sqlalchemy.engine.base.Engine SELECT recipe_list.id, recipe_list.name 
FROM recipe_list
2020-12-01 15:32:52,282 INFO sqlalchemy.engine.base.Engine {}
(1, 'Tuscan Pork Sausage & Pepper Spaghetti')
(2, 'Sweet Potato Fajitas')
(3, 'Chicken Sausage, Sweet Potato & Kale Soup')
(4, 'Pork Sausage & Roasted Bell Pepper Pasta')
